In [1]:
import pandas as pd
import numpy as np

In [2]:
path_load = '../T1DiabetesGranada/'
path_to_save = 'Outputs/'
file = 'Glucose_measurements.csv'
df_glucose = pd.read_csv(path_load + file, dtype={'Measurement' : np.short})
df_glucose.head()

,Patient_ID,Measurement_date,Measurement_time,Measurement
0,LIB193263,2020-06-09,19:08:00,99
1,LIB193263,2020-06-09,19:23:00,92
2,LIB193263,2020-06-09,19:38:00,86
3,LIB193263,2020-06-09,19:53:00,85
4,LIB193263,2020-06-09,20:08:00,85


In [3]:
df_glucose.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22671708 entries, 0 to 22671707
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Patient_ID        object
 1   Measurement_date  object
 2   Measurement_time  object
 3   Measurement       int16 
dtypes: int16(1), object(3)
memory usage: 562.2+ MB


# Sort the dataset by Patient_ID, date and time

In [4]:
df_glucose = df_glucose.sort_values(by=['Patient_ID', 'Measurement_date', 'Measurement_time'])
df_glucose.reset_index(inplace=True, drop=True)
df_glucose.head()

,Patient_ID,Measurement_date,Measurement_time,Measurement
0,LIB193263,2020-06-09,19:08:00,99
1,LIB193263,2020-06-09,19:23:00,92
2,LIB193263,2020-06-09,19:38:00,86
3,LIB193263,2020-06-09,19:53:00,85
4,LIB193263,2020-06-09,20:08:00,85


In [5]:
df_glucose['Measurement_date'].min()

'2018-01-06'

# Search duplicated rows

In [6]:
dup = df_glucose.duplicated()
dup.value_counts()

False    22671708
dtype: int64

No duplicated rows

# Structure where storage the final dataframe of the preprocessing

In [7]:
struct_cols = ['Patient_ID',
               'Days with measures',
               'Measurements', 'Average',
               '0000 - 0600 Total', '0000 - 0600 Hypo', '0000 - 0600 Hyper', '0000 - 0600 Average', '0000 - 0600 STD', '0000 - 0600 MIN', '0000 - 0600 MAX',
               '0600 - 1300 Total', '0600 - 1300 Hypo', '0600 - 1300 Hyper', '0600 - 1300 Average', '0600 - 1300 STD', '0600 - 1300 MIN', '0600 - 1300 MAX',
               '1300 - 1600 Total', '1300 - 1600 Hypo', '1300 - 1600 Hyper', '1300 - 1600 Average', '1300 - 1600 STD', '1300 - 1600 MIN', '1300 - 1600 MAX',
               '1600 - 2000 Total', '1600 - 2000 Hypo', '1600 - 2000 Hyper', '1600 - 2000 Average', '1600 - 2000 STD', '1600 - 2000 MIN', '1600 - 2000 MAX',
               '2000 - 2400 Total', '2000 - 2400 Hypo', '2000 - 2400 Hyper', '2000 - 2400 Average', '2000 - 2400 STD', '2000 - 2400 MIN', '2000 - 2400 MAX'
               ] # Columns on the dataframe to return

# Process dataframe

In [9]:
# Creating the dataframe
df_for_clustering = pd.DataFrame(columns=struct_cols)

morning_0600 =  '06:00:00'
noon_1300 = '13:00:00'
pre_lunch_1600 = '16:00:00'
night_2000 = '20:00:00'

measures = 0    # Total amount of measures
sum = 0         # Total accumulator of glucose measures
hmd = 1         # How many days with measures. Start in one because when star to read the first line it is already the first day of measures

first_measure = 0
first_sum = 0
first_average = 0
first_hypo = 0
first_hyper = 0

second_measure = 0
second_sum = 0
second_average = 0
second_hypo = 0
second_hyper = 0

third_measure = 0
third_sum = 0
third_average = 0
third_hypo = 0
third_hyper = 0

fourth_measure = 0
fourth_sum = 0
fourth_average = 0
fourth_hypo = 0
fourth_hyper = 0

fifth_measure = 0
fifth_sum = 0
fifth_average = 0
fifth_hypo = 0
fifth_hyper = 0

midnight_to_6_values = np.array([])
morning_to_13_values = np.array([])
noon_to_16_values = np.array([])
pre_lunch_to_20_values = np.array([])
night_to_24_values = np.array([])

current_patient = df_glucose.loc[0, 'Patient_ID'] # The first patient of the dataset
current_date = df_glucose.loc[0, 'Measurement_date'] # The first date of the dataset
current_glucose = df_glucose.loc[0, 'Measurement'] # The first measurement of the dataset

for i in range (0,len(df_glucose.index)):
    # Current patient is the same that last one. Although the first iteration it will be always true, and the value
    # assign is the same as the one already have, this is the best option to start the algorithm.
    if current_patient == df_glucose.loc[i, 'Patient_ID']:
        # After doing the new assignments of dates and glucose
        if current_date != df_glucose.loc[i, 'Measurement_date']: # New day measure
            hmd += 1 # Increase the number of days with measures

        current_date = df_glucose.loc[i, 'Measurement_date']
        current_time = df_glucose.loc[i, 'Measurement_time']
        current_glucose = df_glucose.loc[i]['Measurement']

    #END IF
    else: # New patient
        # Update new row in the result dataframe
        if first_measure != 0: # There is at least one measure in the first range
            first_average = first_sum/first_measure

        if second_measure != 0: # There is at least one measure in the second range
            second_average = second_sum/second_measure

        if third_measure != 0: # There is at least one measure in the third range
            third_average = third_sum/third_measure

        if fourth_measure != 0: # There is at least one measure in the fourth range
            fourth_average = fourth_sum/fourth_measure

        if fifth_measure != 0: # There is at least one measure in the fifth range
            fifth_average = fifth_sum/fifth_measure

        struct_dic = {'Patient_ID' : current_patient,
                  'Days with measures' : hmd,
                  'Measurements' : measures,
                  'Average' : sum/measures ,

                  '0000 - 0600 Total' : first_measure,
                  '0000 - 0600 Hypo' : first_hypo,
                  '0000 - 0600 Hyper' : first_hyper,
                  '0000 - 0600 Average' : first_average,
                  '0000 - 0600 STD': midnight_to_6_values.std() if midnight_to_6_values.size > 0 else float('NaN'),
                  '0000 - 0600 MIN': midnight_to_6_values.min() if midnight_to_6_values.size > 0 else float('NaN'),
                  '0000 - 0600 MAX': midnight_to_6_values.max() if midnight_to_6_values.size > 0 else float('NaN'),

                  '0600 - 1300 Total' : second_measure,
                  '0600 - 1300 Hypo' : second_hypo,
                  '0600 - 1300 Hyper' : second_hyper,
                  '0600 - 1300 Average' : second_average,
                  '0600 - 1300 STD': morning_to_13_values.std() if morning_to_13_values.size > 0 else float('NaN'),
                  '0600 - 1300 MIN': morning_to_13_values.min() if morning_to_13_values.size > 0 else float('NaN'),
                  '0600 - 1300 MAX': morning_to_13_values.max() if morning_to_13_values.size > 0 else float('NaN'),

                  '1300 - 1600 Total' : third_measure,
                  '1300 - 1600 Hypo' : third_hypo,
                  '1300 - 1600 Hyper' : third_hyper,
                  '1300 - 1600 Average' : third_average,
                  '1300 - 1600 STD':  noon_to_16_values.std() if noon_to_16_values.size > 0 else float('NaN'),
                  '1300 - 1600 MIN': noon_to_16_values.min() if noon_to_16_values.size > 0 else float('NaN'),
                  '1300 - 1600 MAX': noon_to_16_values.max() if noon_to_16_values.size > 0 else float('NaN'),

                  '1600 - 2000 Total' : fourth_measure,
                  '1600 - 2000 Hypo' : fourth_hypo,
                  '1600 - 2000 Hyper' : fourth_hyper,
                  '1600 - 2000 Average' : fourth_average,
                  '1600 - 2000 STD':  pre_lunch_to_20_values.std() if pre_lunch_to_20_values.size > 0 else float('NaN'),
                  '1600 - 2000 MIN': pre_lunch_to_20_values.min() if pre_lunch_to_20_values.size > 0 else float('NaN'),
                  '1600 - 2000 MAX': pre_lunch_to_20_values.max() if pre_lunch_to_20_values.size > 0 else float('NaN'),

                  '2000 - 2400 Total' : fifth_measure,
                  '2000 - 2400 Hypo' : fifth_hypo,
                  '2000 - 2400 Hyper' : fifth_hyper,
                  '2000 - 2400 Average' : fifth_average,
                  '2000 - 2400 STD':  night_to_24_values.std() if night_to_24_values.size > 0 else float('NaN'),
                  '2000 - 2400 MIN': night_to_24_values.min() if night_to_24_values.size > 0 else float('NaN'),
                  '2000 - 2400 MAX': night_to_24_values.max() if night_to_24_values.size > 0 else float('NaN')
                  }

        # Creating a new series
        series_values = pd.Series(struct_dic)

        # Add new row to the dataframe result
        df_for_clustering = pd.concat([pd.DataFrame([series_values]), df_for_clustering])

        print('------------ ' + current_patient + ' Index: ' + str(i) +  ' END------')

        # Reset the values of the temporal variables
        measures = 0
        sum = 0
        hmd = 1

        first_measure = 0
        first_sum = 0
        first_average = 0
        first_hypo = 0
        first_hyper = 0

        second_measure = 0
        second_sum = 0
        second_average = 0
        second_hypo = 0
        second_hyper = 0

        third_measure = 0
        third_sum = 0
        third_average = 0
        third_hypo = 0
        third_hyper = 0

        fourth_measure = 0
        fourth_sum = 0
        fourth_average = 0
        fourth_hypo = 0
        fourth_hyper = 0

        fifth_measure = 0
        fifth_sum = 0
        fifth_average = 0
        fifth_hypo = 0
        fifth_hyper = 0

        midnight_to_6_values = np.array([])
        morning_to_13_values = np.array([])
        noon_to_16_values = np.array([])
        pre_lunch_to_20_values = np.array([])
        night_to_24_values = np.array([])

        current_patient = df_glucose.loc[i, 'Patient_ID']
        current_date = df_glucose.loc[i, 'Measurement_date']
        current_time = df_glucose.loc[i, 'Measurement_time']
        current_glucose = df_glucose.loc[i]['Measurement']

        print('------------ ' + current_patient + ' Index: ' + str(i) +  ' START ------')
    #END ELSE

    # The process to accumulate all the values. It is made always, if it came from the "if" is the same day and the variables will have values, if
    # came from the else it will begin a new patient with the variables reset it.
    if current_time < morning_0600: # Time between 00:00 and 6:00
        midnight_to_6_values = np.append(midnight_to_6_values, current_glucose)
        first_measure += 1
        first_sum += current_glucose
        if current_glucose < 70: # Hypoglycemic value
            first_hypo += 1
        elif current_glucose > 180: # Hyperglycemic value
            first_hyper += 1

    elif morning_0600 <= current_time < noon_1300: # Time between 06:00 and 13:00
        morning_to_13_values = np.append(morning_to_13_values, current_glucose)
        second_measure += 1
        second_sum += current_glucose
        if current_glucose < 70: # Hypoglycemic value
            second_hypo += 1
        elif current_glucose > 180: # Hyperglycemic value
            second_hyper += 1

    elif noon_1300 <= current_time < pre_lunch_1600: # Time between 13:00 and 16:00
        noon_to_16_values = np.append(noon_to_16_values, current_glucose)
        third_measure += 1
        third_sum += current_glucose
        if current_glucose < 70: # Hypoglycemic value
            third_hypo += 1
        elif current_glucose > 180: # Hyperglycemic value
            third_hyper += 1

    elif pre_lunch_1600 <= current_time < night_2000: # Time between 16:00 and 20:00
        pre_lunch_to_20_values = np.append(pre_lunch_to_20_values, current_glucose)
        fourth_measure += 1
        fourth_sum += current_glucose
        if current_glucose < 70: # Hypoglycemic value
            fourth_hypo += 1
        elif current_glucose > 180: # Hyperglycemic value
            fourth_hyper += 1

    elif night_2000 <= current_time: # Time between 20:00 and 24:00
        night_to_24_values = np.append(night_to_24_values, current_glucose)
        fifth_measure += 1
        fifth_sum += current_glucose
        if current_glucose < 70: # Hypoglycemic value
            fifth_hypo += 1
        elif current_glucose > 180: # Hyperglycemic value
            fifth_hyper += 1

    # General stats
    measures += 1
    sum += current_glucose
#END FOR

# Adding the last patient
if first_measure != 0: # There is at least one measure in the first range
    first_average = first_sum/first_measure

if second_measure != 0: # There is at least one measure in the second range
    second_average = second_sum/second_measure

if third_measure != 0: # There is at least one measure in the third range
    third_average = third_sum/third_measure

if fourth_measure != 0: # There is at least one measure in the fourth range
    fourth_average = fourth_sum/fourth_measure

if fifth_measure != 0: # There is at least one measure in the fifth range
    fifth_average = fifth_sum/fifth_measure

struct_dic = {'Patient_ID' : current_patient,
          'Days with measures' : hmd,
          'Measurements' : measures,
          'Average' : sum/measures ,

          '0000 - 0600 Total' : first_measure,
          '0000 - 0600 Hypo' : first_hypo,
          '0000 - 0600 Hyper' : first_hyper,
          '0000 - 0600 Average' : first_average,
          '0000 - 0600 STD': midnight_to_6_values.std() if midnight_to_6_values.size > 0 else float('NaN'),
          '0000 - 0600 MIN': midnight_to_6_values.min() if midnight_to_6_values.size > 0 else float('NaN'),
          '0000 - 0600 MAX': midnight_to_6_values.max() if midnight_to_6_values.size > 0 else float('NaN'),

          '0600 - 1300 Total' : second_measure,
          '0600 - 1300 Hypo' : second_hypo,
          '0600 - 1300 Hyper' : second_hyper,
          '0600 - 1300 Average' : second_average,
          '0600 - 1300 STD': morning_to_13_values.std() if morning_to_13_values.size > 0 else float('NaN'),
          '0600 - 1300 MIN': morning_to_13_values.min() if morning_to_13_values.size > 0 else float('NaN'),
          '0600 - 1300 MAX': morning_to_13_values.max() if morning_to_13_values.size > 0 else float('NaN'),

          '1300 - 1600 Total' : third_measure,
          '1300 - 1600 Hypo' : third_hypo,
          '1300 - 1600 Hyper' : third_hyper,
          '1300 - 1600 Average' : third_average,
          '1300 - 1600 STD':  noon_to_16_values.std() if noon_to_16_values.size > 0 else float('NaN'),
          '1300 - 1600 MIN': noon_to_16_values.min() if noon_to_16_values.size > 0 else float('NaN'),
          '1300 - 1600 MAX': noon_to_16_values.max() if noon_to_16_values.size > 0 else float('NaN'),

          '1600 - 2000 Total' : fourth_measure,
          '1600 - 2000 Hypo' : fourth_hypo,
          '1600 - 2000 Hyper' : fourth_hyper,
          '1600 - 2000 Average' : fourth_average,
          '1600 - 2000 STD':  pre_lunch_to_20_values.std() if pre_lunch_to_20_values.size > 0 else float('NaN'),
          '1600 - 2000 MIN': pre_lunch_to_20_values.min() if pre_lunch_to_20_values.size > 0 else float('NaN'),
          '1600 - 2000 MAX': pre_lunch_to_20_values.max() if pre_lunch_to_20_values.size > 0 else float('NaN'),

          '2000 - 2400 Total' : fifth_measure,
          '2000 - 2400 Hypo' : fifth_hypo,
          '2000 - 2400 Hyper' : fifth_hyper,
          '2000 - 2400 Average' : fifth_average,
          '2000 - 2400 STD':  night_to_24_values.std() if night_to_24_values.size > 0 else float('NaN'),
          '2000 - 2400 MIN': night_to_24_values.min() if night_to_24_values.size > 0 else float('NaN'),
          '2000 - 2400 MAX': night_to_24_values.max() if night_to_24_values.size > 0 else float('NaN')
          }

# Creating a new series
series_values = pd.Series(struct_dic)
df_for_clustering = pd.concat([pd.DataFrame([series_values]), df_for_clustering])

df_for_clustering

------------ LIB193263 Index: 60098 END------
------------ LIB193264 Index: 60098 START ------
------------ LIB193264 Index: 86885 END------
------------ LIB193265 Index: 86885 START ------
------------ LIB193265 Index: 133461 END------
------------ LIB193266 Index: 133461 START ------
------------ LIB193266 Index: 188769 END------
------------ LIB193267 Index: 188769 START ------
------------ LIB193267 Index: 244270 END------
------------ LIB193268 Index: 244270 START ------
------------ LIB193268 Index: 285351 END------
------------ LIB193269 Index: 285351 START ------
------------ LIB193269 Index: 357232 END------
------------ LIB193272 Index: 357232 START ------
------------ LIB193272 Index: 360668 END------
------------ LIB193273 Index: 360668 START ------
------------ LIB193273 Index: 377310 END------
------------ LIB193274 Index: 377310 START ------
------------ LIB193274 Index: 389114 END------
------------ LIB193276 Index: 389114 START ------
------------ LIB193276 Index: 3910

,ID,Days with measures,Measurements,Average,0000 - 0600 Total,0000 - 0600 Hypo,0000 - 0600 Hyper,0000 - 0600 Average,0000 - 0600 STD,0000 - 0600 MIN,...,1600 - 2000 STD,1600 - 2000 MIN,1600 - 2000 MAX,2000 - 2400 Total,2000 - 2400 Hypo,2000 - 2400 Hyper,2000 - 2400 Average,2000 - 2400 STD,2000 - 2400 MIN,2000 - 2400 MAX
0,LIB194166,315,33372,131.52613,8194,252,581,121.954601,37.405866,53.0,...,47.847068,53.0,358.0,5468,163,722,132.890271,40.717756,53.0,302.0
0,LIB194165,259,23863,179.308008,6017,222,3473,195.923716,73.126044,46.0,...,77.489332,50.0,457.0,3649,251,1270,158.545629,68.050734,40.0,403.0
0,LIB194164,632,58350,152.404473,13964,901,3841,147.788886,56.172238,43.0,...,52.405727,53.0,347.0,9520,225,3043,157.706303,50.48212,49.0,342.0
0,LIB194163,192,17755,151.133202,4367,227,1335,153.449737,63.242676,40.0,...,69.318881,53.0,500.0,2849,108,748,148.118989,55.314128,53.0,417.0
0,LIB194162,361,33440,149.587081,8181,751,2200,146.611417,65.142357,48.0,...,59.63904,43.0,469.0,5536,252,1823,157.267522,61.212698,45.0,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,LIB193267,647,55501,181.668474,13807,390,6230,182.237126,75.258731,40.0,...,67.229402,40.0,437.0,8325,92,4627,199.35039,76.526081,53.0,488.0
0,LIB193266,500,55308,155.20196,12286,357,3751,154.638206,57.57169,40.0,...,56.991521,52.0,500.0,8413,57,3042,167.311661,60.369626,40.0,462.0
0,LIB193265,581,46576,133.811298,11263,2408,1493,119.460091,61.74354,40.0,...,67.746997,40.0,500.0,6268,736,1136,135.097479,66.939892,41.0,500.0
0,LIB193264,326,26787,149.991675,6733,428,2333,160.231695,65.46448,40.0,...,56.532564,40.0,349.0,4071,348,989,140.503562,57.58675,41.0,353.0


# Save preprocess dataset

In [12]:
df_for_clustering.to_csv(path_to_save+'To clustering - V5.0.csv', index=False)